In [1]:
import cv2
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

#如果没有安装opencv的话，这里可以替换成matplotlib读取图片
img = cv2.imread("imori.jpg")
img_noise = cv2.imread("imori_noise.jpg")
img_dark = cv2.imread("imori_dark.jpg")
#使用matplotlib输出图片，所以要将BGR格式预先转为RBG格式
img = img[:,:,[2,1,0]]
img_noise = img_noise[:,:,[2,1,0]]
img_dark = img_dark[:,:,[2,1,0]]

In [2]:
#灰度化
def gray(img):
    newimg = img[:,:,0]*0.2126 + img[:,:,1]*0.7152 + img[:,:,2]*0.0722
    return newimg

In [ ]:
# 均值滤波器

# 对图像四周进行0填充,填充宽度为s
def padding(img, s=1):
    #先加行
    rows = np.zeros((s, img.shape[1], img.shape[2]), dtype="uint8")
    #再加列，这时候列长已经扩充 2*s,也就是（128×128） -> (129×129)
    columns = np.zeros((img.shape[0]+(2*s),s,img.shape[2]),dtype="uint8")
    #拼接
    img = np.vstack((rows,img,rows))
    img = np.hstack((columns,img,columns))
    return img

def avgFilter(area):
    #拉成一条，并且转置为横向
    area = area.reshape(-1,3).T
    #进行排序后取出中间值
    newk = area.mean(axis = 1)
    return newk



In [ ]:
#进行3×3滤波
def getarea(img,padding,method):
    #建立一个新的数组，该数组用于保存计算的高斯滤波值，所以要去掉padding
    #判断是单通道还是多通道(灰度or彩色)
    if len(img.shape)>2:
        result = np.zeros([img.shape[0]-2*padding,
                      img.shape[1]-2*padding,
                      img.shape[2]],dtype="uint8")
    else:
        result = np.zeros([img.shape[0]-2*padding,
                      img.shape[1]-2*padding],dtype="uint8")
    
    for i in range(int(img.shape[0]-2)):
        for j in range(int(img.shape[1]-2)):
            area = img[i:i+3,j:j+3]
            #均值滤波器
            if method=="average":
                result[i,j] = avgFilter(area)
            #运动滤波器
            if method=="motion":
                result[i,j] = motFilter(area)
            #max-min滤波器
            if method=="maxmin":
                result[i,j] = maxminFilter(area)
            #差分滤波器(纵向)
            if method=="differential_v":
                result[i,j] = diffFilter(area,True)
            #差分滤波器(横向)
            if method=="differential_h":
                result[i,j] = diffFilter(area,False)
            #Sobel滤波器(纵向)
            if method=="sobel_v":
                result[i,j] = sobelFilter(area,True)
            #Sobel滤波器(横向)
            if method=="sobel_h":
                result[i,j] = sobelFilter(area,False)   
            #Prewitt录波器(纵向)
            if method=="prewitt_v":
                result[i,j] = prewittFilter(area,True)
            #Prewitt录波器(横向)
            if method=="prewitt_h":
                result[i,j] = prewittFilter(area,False) 
            #拉普拉斯滤波器
            if method=="laplacian":
                result[i,j] = laplacianFilter(area)   
            #Emboss滤波器
            if method=="emboss":
                result[i,j] = embossFilter(area) 
            #高斯滤波器
            if(method=="gaussian"):
                result[i,j] = gaussFilter(area)
            #中值滤波器
            if(method=="median"):
                result[i,j] = medianFilter(area)
    return result

In [ ]:
imgshow = plt.imshow(img)
plt.show()

img1 = img.copy()
img1 = padding(img1,1)
img1 = getarea(img1,1,"average")
imgshow = plt.imshow(img1)